In [73]:
!pip install findspark pyspark

Looking in indexes: https://edge.artifactory.ouroath.com:4443/artifactory/api/pypi/pypi-mirror/simple

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [74]:
import os
import findspark
import pyspark

In [75]:
spark_home = os.environ.get('SPARK_HOME', '/opt/homebrew/Cellar/apache-spark/3.5.4/libexec')

In [76]:
findspark.init()

from pyspark.sql import SparkSession

In [77]:
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

In [78]:
spark

# Answer 1

v3.5.4


In [79]:
data_url = "/Users/ajain17/Documents/Developer/de-zoomcamp-homework-2025/hw5/data/yellow_tripdata_2024-10.parquet"
df = spark.read.parquet(data_url)
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [80]:
ans2_df = df.repartition(4).write.mode("overwrite").parquet("./hw5/answers/ans2-output/")

In [81]:
!ls -ltrgh ./hw5/answers/ans2-output/

total 194880
-rw-r--r--@ 1 staff    24M Feb 26 21:00 part-00000-d1c9e3a4-55ee-45ba-a37c-1a00cad54754-c000.snappy.parquet
-rw-r--r--@ 1 staff    24M Feb 26 21:00 part-00001-d1c9e3a4-55ee-45ba-a37c-1a00cad54754-c000.snappy.parquet
-rw-r--r--@ 1 staff    24M Feb 26 21:00 part-00002-d1c9e3a4-55ee-45ba-a37c-1a00cad54754-c000.snappy.parquet
-rw-r--r--@ 1 staff    24M Feb 26 21:00 part-00003-d1c9e3a4-55ee-45ba-a37c-1a00cad54754-c000.snappy.parquet
-rw-r--r--@ 1 staff     0B Feb 26 21:00 _SUCCESS


# Answer 2

25MB


In [82]:
# Question 3
# How many taxi trips were there on the 15th of October? Consider only trips that started on the 15th of October.
from pyspark.sql.functions import col, to_date

ans3 = df.filter(to_date(col("tpep_pickup_datetime")) == "2024-10-15").count()
ans3

128893

# Answer 3

128893


In [96]:
# Question 4: Longest trip
# What is the length of the longest trip in the dataset in hours?
from pyspark.sql.functions import col, unix_timestamp
trip_duration_df = df.withColumn("trip_duration_hours", (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600)
ans4_df = trip_duration_df.select("PULocationID", "DOLocationID", "tpep_pickup_datetime","tpep_dropoff_datetime", "trip_duration_hours").sort(col("trip_duration_hours").desc())
ans4_df.show(5)
# ans4 = trip_duration_df.select("trip_duration_hours").sort(col("trip_duration_hours").desc()).first()[0]
ans4 = ans4_df.select("trip_duration_hours").first()[0]
ans4

+------------+------------+--------------------+---------------------+-------------------+
|PULocationID|DOLocationID|tpep_pickup_datetime|tpep_dropoff_datetime|trip_duration_hours|
+------------+------------+--------------------+---------------------+-------------------+
|          48|         265| 2024-10-16 13:03:49|  2024-10-23 07:40:53| 162.61777777777777|
|         100|         100| 2024-10-03 18:47:25|  2024-10-09 18:06:55|            143.325|
|         186|         226| 2024-10-22 16:00:55|  2024-10-28 09:46:33| 137.76055555555556|
|         116|         163| 2024-10-18 09:53:32|  2024-10-23 04:43:37| 114.83472222222223|
|         122|         138| 2024-10-21 00:36:24|  2024-10-24 18:30:18|  89.89833333333333|
+------------+------------+--------------------+---------------------+-------------------+
only showing top 5 rows



162.61777777777777

# Answer 4

162


# Answer 5

Spark’s User Interface which shows the application's dashboard runs on which local port?

`4040`


In [100]:
# Question 6: Least frequent pickup location zone
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

# Load the zone lookup data into a temp view in Spark:
taxi_zone_lookup_df = spark.read.csv("/Users/ajain17/Documents/Developer/de-zoomcamp-homework-2025/hw5/data/taxi_zone_lookup.csv", header=True)
taxi_zone_lookup_df.createOrReplaceTempView("taxi_zone_lookup")
# Temp view for yellow_tripdata
df.createOrReplaceTempView("yellow_tripdata")
ans6_df = spark.sql("SELECT Zone as pickup_zone, count(*) as count FROM yellow_tripdata yt JOIN taxi_zone_lookup t ON yt.PULocationID = t.LocationID GROUP BY 1 ORDER BY count")
ans6_df.show(truncate=False, n=5)
ans6 = ans6_df.select("pickup_zone").first()[0]
ans6


+---------------------------------------------+-----+
|pickup_zone                                  |count|
+---------------------------------------------+-----+
|Governor's Island/Ellis Island/Liberty Island|1    |
|Rikers Island                                |2    |
|Arden Heights                                |2    |
|Jamaica Bay                                  |3    |
|Green-Wood Cemetery                          |3    |
+---------------------------------------------+-----+
only showing top 5 rows



"Governor's Island/Ellis Island/Liberty Island"

# Answer 6

`Governor's Island/Ellis Island/Liberty Island`
